In [ ]:
#@title # **下載**
# 初始化
from google.colab import drive
import os, json, urllib.parse, re, requests
if not os.path.exists("/content/DL"):
  !mkdir -p /content/DL
  !rm -rf /content/sample_data
  !apt-get install aria2
  !pip install uv
  !uv pip install gigafile --system

%cd /content/DL
link = "" #@param{type:"string"}
from_direct_link = False #@param{type:"boolean"}
from_torrent = False #@param{type:"boolean"}
from_gigafile = False #@param{type:"boolean"}
from_pixeldrain = False #@param{type:"boolean"}
from_gofile = False #@param{type:"boolean"}

if from_torrent:
  !aria2c --summary-interval=10 --seed-time=0 --allow-overwrite=true "{link}"

if from_direct_link:
  !aria2c --summary-interval=10 -c -x 16 -k 1M -s 16 "{link}"

if from_gigafile:
  !gfile download "{link}"

if from_pixeldrain:
  url_check = re.match('https://pixeldrain.com/u/', link)
  if url_check:
    pd_id = link[url_check.span()[1]: len(link)]
    bypass = 'https://pd.cybar.xyz/' + pd_id
    !aria2c --summary-interval=10 -c -x 16 -k 1M -s 16 "{bypass}"
  else:
    print("URL格式錯誤！")
    print("正確格式應為`https://pixeldrain.com/u/XXXXXXXX`")

if from_gofile:
  if not os.path.exists("/content/gofile-downloader"):
    %cd /content
    # 安裝gofile downloader
    # https://github.com/ltsdw/gofile-downloader
    !git clone https://github.com/ltsdw/gofile-downloader.git
    %cd gofile-downloader
    !uv pip install -r requirements.txt --system
    %cd /content/DL
  !python /content/gofile-downloader/gofile-downloader.py "{link}"


print("\nDone!")

---
# **上傳**

In [ ]:
#@title # **上傳到Buzzheavier**

filename = "" #@param{type:"string"}

BuzzUpUrl = "https://w.buzzheavier.com/" + urllib.parse.quote(filename)
BuzzUp = !curl -#o - -T "$filename" "$BuzzUpUrl" | cat
try:
  if int(json.loads(BuzzUp[-1]).get('code')) == 201:
    print("下載網址： https://buzzheavier.com/", json.loads(BuzzUp[-1]).get('data').get('id'), sep='')
except:
  print("下面data中的id放在 `https://buzzheavier.com/\"這裡\"` 就是下載網址\n如果沒有id那就是出錯啦\n", BuzzUp[-1])

In [ ]:
#@title # **上傳到Pixeldrain**
#@markdown **Pixeldrain上限 [20](https://pixeldrain.com/) GB**

#@markdown 現在需要帳號ㄌ！

filename = "" #@param{type:"string"}
pd_api_key = "" #@param{type:"string"}

if pd_api_key == "":
  print("請填入API密鑰！")
  print("https://pixeldrain.com/user/api_keys")
else:
  pdupload = !curl -T "{filename}" -u :$pd_api_key https://pixeldrain.com/api/file/
  res = json.loads(pdupload[0])
  print("下載網址： https://pixeldrain.com/u/", res.get('id'), sep='')

In [ ]:
#@title # **上傳到Gofile**

filename = "" #@param{type:"string"}
# 就是顯示更詳細的資料
dev_mode = False #@param{type:"boolean"}

# !curl -X GET 'https://api.gofile.io/servers'
server = !curl -X GET 'https://api.gofile.io/servers'
good_server = json.loads(server[0]).get('data').get('servers')[0].get('name')
go_url = f"https://{good_server}.gofile.io/contents/uploadfile"
# !curl -X POST https://store1.gofile.io/contents/uploadfile -F "file=@{filename}"
go_up = !curl -X POST $go_url -F "file=@{filename}"
print('下載網址：', json.loads(go_up[0]).get('data').get('downloadPage'))

if dev_mode:
  gofile_token = json.loads(go_up[0]).get('data').get('guestToken')
  parentID = json.loads(go_up[0]).get('data').get('parentFolder')
  print('guestToken： ', gofile_token)
  print('parentFolder: ', parentID)
  print('server: ', good_server)
  # !curl -H "Authorization: Bearer " -X POST https://store.gofile.io/contents/uploadfile -F "file=@" -F "folderId="

In [ ]:
#@title # **上傳到Gigafile**

filename = "" #@param{type:"string"}

!gfile -p -n 24 upload "{filename}"

In [ ]:
#@title # **上傳到Gofile.to**

filename = "" #@param{type:"string"}
SF = !curl -F "file=@{filename}" https://gofile.to/api/v1/upload
print("下載網址：", json.loads(SF[0]).get('data').get('file').get('url').get('short'))

In [ ]:
#@title # 上傳到Google Drive
#@markdown **資料夾位置記得加 "/"**

google_drive_path = "ColabDownloads/" #@param {type:"string"}

if not os.path.exists('/content/gdrive'):
  drive.mount('/content/gdrive')

!rsync -avh --progress . "/content/gdrive/MyDrive/"{output_path}

print("\033[32mDone!\033[0m")

In [ ]:
#@title # **上傳到catbox**
#@markdown **catbox上限 [200](https://catbox.moe/) MB**

filename = "" #@param{type:"string"}

!curl -F "reqtype=fileupload" -F "fileToUpload=@{filename}" https://catbox.moe/user/api.php

---
# 其他

In [ ]:
#@title # 👇 **移除下載檔案 (清理一下colab)**
!rm -rf /content/DL/*
%cd /content/DL
print("乾淨ㄌ")

In [ ]:
# 自行發揮

---
# 以下小工具


In [ ]:
#@title # Colab終端機
#@markdown [colab-xterm](https://github.com/InfuseAI/colab-xterm)
!pip install colab-xterm
%load_ext colabxterm
%xterm

In [ ]:
#@title # Telegram 工具
#@markdown [tdl](https://github.com/iyear/tdl) [文件](https://docs.iyear.me/tdl/)
!curl -sSL https://docs.iyear.me/tdl/install.sh | sudo bash
# upload tdl
# unzip tdl.zip -d ~/.tdl

In [ ]:
#@title # Mega帳號產生器
#@markdown 使用 [MEGAcmd](https://github.com/meganz/MEGAcmd) 和 [mail.tm](https://mail.tm/) API 產生帳號
# 使用方法 https://github.com/meganz/MEGAcmd/blob/master/UserGuide.md
import random, string, json, threading, re


def generate_account(domain, username_length=8, password_length=12):
    username = ''.join(random.choices(string.ascii_lowercase + string.digits, k=username_length))
    password_characters = string.ascii_letters + string.digits + "!@#$%^&*()"
    password = ''.join(random.choices(password_characters, k=password_length))
    address = f"{username}@{domain}"
    return {"address": address, "password": password}


def wtfmega():
  # 防止卡住，我想不到更好的方法，先這樣處理
  !mega-ls


%cd /content
# 安裝 MEGAcmd
# 目前我看colab的系統是Ubuntu 22.04 amd64
# 如果之後有需要做更改可以看一下系統資訊
# !cat /etc/os-release
# !uname -m
!wget https://mega.nz/linux/repo/xUbuntu_22.04/amd64/megacmd-xUbuntu_22.04_amd64.deb && sudo apt install "$PWD/megacmd-xUbuntu_22.04_amd64.deb"
t = threading.Thread(target=wtfmega)
t.start()

domain = !curl -X GET https://api.mail.tm/domains | jq -r '."hydra:member"[0].domain'
account_info = generate_account(domain[-1])
json_data = json.dumps(account_info)
curl_cmd = f'''curl -X POST https://api.mail.tm/accounts -H "Content-Type: application/json" -d '{json_data}' '''
!$curl_cmd
curl_cmd = f'''curl -X POST https://api.mail.tm/token -H "Content-Type: application/json" -d '{json_data}' '''
tmtoken = !$curl_cmd | jq -r .token
!mega-signup "{account_info.get('address')}" "{account_info.get('password')}"
maildl = !curl -H "Authorization: Bearer {tmtoken[-1]}" -X GET https://api.mail.tm/messages -H "Content-Type: application/json" | jq -r '."hydra:member"[0].downloadUrl'
!curl -H "Authorization: Bearer {tmtoken[-1]}" -X GET https://api.mail.tm"{maildl[-1]}" -o mail.txt
with open('mail.txt', 'r') as file:
    content = file.read()

confirm_link = re.search(r'https://mega\.nz/#confirm\S+', content)

if confirm_link:
    confirm_url = confirm_link.group()
else:
    print("找不到確認連結")
!mega-confirm $confirm_url "{account_info.get('address')}" "{account_info.get('password')}"
!mega-login "{account_info.get('address')}" "{account_info.get('password')}"
print(f"\n\n\n\n\nMega帳號： {account_info.get('address')}\nMega密碼： {account_info.get('password')}\n\n\n\n\n")
# 如果沒有意外可以使用：
# 上傳 !mega-put 檔案
# 下載 !mega-get link
# 分享(需輸入Yes) !mega-export -a 檔案 或 !echo Yes | mega-export -a 檔案

In [ ]:
#@title # Gdrive
#@markdown [gdrive](https://github.com/glotlabs/gdrive)
%cd /content
!wget https://github.com/glotlabs/gdrive/releases/download/3.9.1/gdrive_linux-x64.tar.gz
!tar zxvf gdrive_linux-x64.tar.gz

# 如何登入自己的帳號？請看以下網址
# https://github.com/glotlabs/gdrive/blob/main/docs/create_google_api_credentials.md

# !./gdrive account import account.tar
# !rm gdrive_linux-x64.tar.gz account.tar
%cd /content/DL
# 下載
# !/content/gdrive files download "id" --recursive
# https://drive.google.com/file/d/"這裡面的就是ID"/view
# 上傳
# !/content/gdrive files upload [OPTIONS] [FILE_PATH]

In [ ]:
#@title # YouTube影片下載
#@markdown [po_token](https://github.com/Brainicism/bgutil-ytdlp-pot-provider)

import json, os
yt_url = "https://youtu.be/dQw4w9WgXcQ" #@param {type:"string"}

#-----之前的程式碼，應該還有研究用途
'''
# [po_token](https://github.com/iv-org/youtube-trusted-session-generator)
import json, os
if not os.path.exists("data.json"):
  !pip install uv
  !uv pip install udocker yt-dlp --system
  !udocker --allow-root install
  # 處理po_token的部分
  gen = !udocker --allow-root run quay.io/invidious/youtube-trusted-session-generator
  # 資料有問題可能是行數出錯ㄌㄏㄏ
  # print(gen)
  visitor_data = gen[22].split(': ')[1]
  po_token = gen[23].split(': ')[1]
  data = {"visitor_data": visitor_data, "po_token": po_token}
  args = "youtube:player-client=web,default;player-skip=webpage,configs;po_token=web+" + po_token + ";visitor_data=" + visitor_data
  json_object = json.dumps(data, indent=4)
  # 把資料存到data.json

  import json
  with open('data.json', 'r') as handle:
      parsed = json.load(handle)
  print(json.dumps(parsed, indent=4))

  with open("data.json", "w") as outfile:
      outfile.write(json_object)
yt_url = "https://youtu.be/dQw4w9WgXcQ"
!yt-dlp --extractor-args "{args}" --ignore-errors --output "%(title)s.%(ext)s" --verbose --merge-output-format mp4 --embed-thumbnail --embed-metadata --embed-info-json --concurrent-fragments 24 "{yt_url}"
# 這裡可以自己DIY，那串不要刪掉，在那串後面DIY
# !yt-dlp --extractor-args "youtube:player-client=web,default;player-skip=webpage,configs;po_token=web+{po_token};visitor_data={visitor_data}"
'''
if not os.path.exists("installed"):
  !fallocate -l 1 installed
  !apt-get install screen
  !pip install uv
  !uv pip install udocker yt-dlp --system
  !uv pip install -U bgutil-ytdlp-pot-provider
  !screen -d -m udocker --allow-root run -p 4416:4416 brainicism/bgutil-ytdlp-pot-provider
  %cd ~
  !git clone --single-branch --branch 0.7.4 https://github.com/Brainicism/bgutil-ytdlp-pot-provider.git
  %cd bgutil-ytdlp-pot-provider/server/
  !npm install -g yarn
  !yarn install --frozen-lockfile
  !npx --yes tsc
  %cd /content

!yt-dlp --ignore-errors --output "%(title)s.%(ext)s" --verbose --merge-output-format mp4 --embed-thumbnail --embed-metadata --embed-info-json --concurrent-fragments 24 "{yt_url}"

In [ ]:
#@title # YouTube直播下載
#@markdown [ytarchive](https://github.com/Kethsar/ytarchive)
%cd /content
!pip install uv
!uv pip install yt-dlp
import threading
!wget https://github.com/Kethsar/ytarchive/releases/download/latest/ytarchive_linux_amd64.zip
!unzip ytarchive_linux_amd64.zip
!rm ytarchive_linux_amd64.zip

yt_url = "" #@param{type:"string"}
mp4 = False #@param{type:"boolean"}
live_chat = False #@param{type:"boolean"}


# 下載直播聊天室
def job():
  !yt-dlp -o "[%(upload_date)s] %(title)s [%(channel)s] (%(id)s)" --write-sub --sub-lang live_chat --skip-download -v --wait-for-video 1 --ignore-config "{yt_url}"


if live_chat:
  t = threading.Thread(target=job)
  t.start()
  if mp4:
    !./ytarchive --add-metadata --vp9 -o "[%(upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(upload_date)s] %(title)s [%(channel)s] (%(id)s)" --wait -t --threads 4 -v --write-description --write-thumbnail "{yt_url}" best
  else:
    !./ytarchive --add-metadata --mkv --vp9 -o "[%(upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(upload_date)s] %(title)s [%(channel)s] (%(id)s)" --wait -t --threads 4 -v --write-description --write-thumbnail "{yt_url}" best
  t.join()
else:
  if mp4:
    !./ytarchive --add-metadata --vp9 -o "[%(upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(upload_date)s] %(title)s [%(channel)s] (%(id)s)" --wait -t --threads 4 -v --write-description --write-thumbnail "{yt_url}" best
  else:
    !./ytarchive --add-metadata --mkv --vp9 -o "[%(upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(upload_date)s] %(title)s [%(channel)s] (%(id)s)" --wait -t --threads 4 -v --write-description --write-thumbnail "{yt_url}" best

In [ ]:
%cd /content
#@title # Firefox瀏覽器
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/Firefox_logo%2C_2019.svg/330px-Firefox_logo%2C_2019.svg.png' height="100" alt="Firefox"/></center>
#@markdown <center><h3><a href="https://github.com/jlesage/docker-firefox">jlesage/firefox</a>, <a href="https://github.com/jlesage/docker-firefox-esr">firefox-esr</a></h3></center>
#@markdown <center>抄來ㄉ <a href="https://colab.research.google.com/github/biplobsd/OneClickRun/blob/master/OneClickRun.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg"></center><br>

import os, tarfile
import urllib.request
from IPython.display import clear_output
import subprocess
import socket
import time

#####################################
USE_FREE_TOKEN = True # 免費token

TOKEN = ""  # 有需要再用，很常出問題
REGION = "JP" # 選擇地區 ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
PORT_FORWARD = "localhost" # 連線方式 ["ngrok", "localhost", "argotunnel"]
TYPE = "firefox" # 版本 ["firefox", "firefox-esr"]
CONFIG_DIRECTORY = "/content/tools/firefox"  # 設定檔位置
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")


from ocr import (
    loadingAn,
    PortForward_wrapper,
    findProcess,
    read_subprocess_output,
    textAn
)

def is_port_open(port):
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.settimeout(2)
            s.connect(("localhost", port))
        return True
    except (socket.timeout, ConnectionRefusedError):
        return False

def in_output(command, output):
    try:
        result = subprocess.run(command.split(), capture_output=True, text=True, check=True)
        return output in result.stdout
    except subprocess.CalledProcessError:
        return False

def check_log_until_line_appears(command, line_to_check):
    process = subprocess.Popen(command, stdout=subprocess.PIPE)
    while True:
        line = process.stdout.readline().decode('utf-8')
        if line_to_check in line:
            return True
        elif line == '':
            return False
        else:
            time.sleep(0.1)

def popen(cmd, description=None):
    try:
        if description:
          clear_output()
          loadingAn()
          textAn(description, 'twg')
        subprocess.Popen(cmd.split()).wait()
    except subprocess.CalledProcessError as e:
        print(f"Error running command: {cmd}")
        print(f"Error message: {e}")
        raise

loadingAn()
os.makedirs(CONFIG_DIRECTORY, exist_ok=True)

if not os.path.exists("/usr/local/bin/udocker"):
    popen('pip install udocker', "正在安裝 udocker ...")
    popen('udocker --allow-root install', '設定 udocker ...')

if not in_output('udocker --allow-root images', f'jlesage/{TYPE}:latest'):
    popen(f'udocker --allow-root pull jlesage/{TYPE}', f"獲取 jlesage/{TYPE} image ...")

if not in_output('udocker --allow-root ps', TYPE):
    popen(f'udocker --allow-root create --name={TYPE} jlesage/{TYPE}', f"創建 {TYPE} container ...")

command = f'udocker --allow-root run -v {CONFIG_DIRECTORY}:/config -p 5800:5800 {TYPE}'.split()
line_to_check = '[supervisor  ] all services started.'

done = True
if (not findProcess(TYPE, '/config/profile')) or (not is_port_open(5800)):
    popen('fuser -k -n tcp 5900')
    clear_output()
    loadingAn()
    textAn("等待所有系統開始運行 ...", 'twg')
    done = check_log_until_line_appears(command, line_to_check)

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
if done:
    server = PortForward_wrapper(
        PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['firefox', '5800', 'http']],
        REGION.lower(),
        [f"{HOME}/.ngrok2/firefox.yml", 58009]
    ).start('firefox', displayB=True)
else:
    print("Error: 請終止執行階段，並重新執行。")


In [ ]:
#@title # Speedtest
#@markdown [SPEEDTEST](https://www.speedtest.net/zh-Hant/apps/cli)
!curl -s https://packagecloud.io/install/repositories/ookla/speedtest-cli/script.deb.sh | sudo bash
!apt-get install speedtest
!echo YES | speedtest